Let's start by importing all the data and libraries needed

In [176]:
import pandas as pd
import numpy as np
import pickle 
from datetime import datetime
from fuzzywuzzy import fuzz

with open('riaa_df.pkl', 'rb') as f:
        riaa_df = pickle.load(f)
with open('hot-100_chart_final.pkl', 'rb') as f:
        rec_df= pickle.load(f)     
with open('song_df.pkl', 'rb') as f:
        song_df = pickle.load(f)

We create a few features from the RIAA data based on the number of gold disc received, the number of platinum and the number of awards for an artists (number of songs with an awards) depending on the RIAA data

In [ ]:
riaa_df["award_datetime"] = riaa_df["award_date"].apply(lambda x: datetime.strptime(x.strip(),"%B %d, %Y"))
song_df["spotify_album_release_date_datetime"] = song_df["spotify_album_release_date"].apply(lambda x: datetime.strptime(x.strip(),"%Y-%m-%d") if len(str(x))>8 else x)
riaa_df["award"] = riaa_df["award"].apply(lambda x: x.strip())
keywords = ["Featuring","Feauring", "featuring", "&",","," X ", " x ","Presents", "From", "Or","And",]
for song in song_df.index:
    date = song_df.loc[song,"spotify_album_release_date_datetime"]
    artist_df = riaa_df[riaa_df["award_datetime"] <= date]
    artist = song_df.loc[song,"artist"]
    last_award_list = list()
    award_num_list = list()
    num_songs_awards = list()
    gold_count = list()
    platinum_count = list()
    
    artist_list = list()
    artist_list.append(artist)
    for keyword in keywords:
        artist_list.extend(artist.split(keyword))
    artist_list = list(set(artist_list))
    
    if len(str(date)) >8:
        for performer in artist_list:
            artist_df1 = artist_df[artist_df["artist"] == performer.strip()].sort_values("award_datetime",ascending=False)
            if len(artist_df1)>0:
                gold_count.append(artist_df1[artist_df1["award"] == "Gold"].count()["award"])
                artist_df1 = artist_df1.sort_values("award_datetime",ascending=False).drop_duplicates("songs")
                num_songs_awards.append(len(artist_df1["songs"].value_counts()))
                platinum_count.append(artist_df1[artist_df1["award"] == "Platinum"].sum()["award_num"])
                last_award_list.append(artist_df1.iloc[0]["award"])
                award_num_list.append(gold_count[-1] + platinum_count[-1])
            else:
                last_award_list.append(None)
                award_num_list.append(0)
                gold_count.append(0)
                platinum_count.append(0)
                num_songs_awards.append(0)
 
        max_index = award_num_list.index(max(award_num_list))
        song_df.loc[song,"last_award_type"] = last_award_list[max_index]
        song_df.loc[song,"award_num"] = award_num_list[max_index]
        song_df.loc[song,"gold_count"] = gold_count[max_index]
        song_df.loc[song,"platinum_count"] = platinum_count[max_index]
        song_df.loc[song,"num_songs_awards"] = num_songs_awards[max_index]
song_df["artist_has_award"] = song_df["award_num"] > 0

We know look at the data for Labels (25 of the labels are involved in 3936 of the 8516 songs...) so we create categorical variables

In [100]:
labels_occurence = list()
for i in np.asarray(song_df["album_label"]):
    if type(i) == str:
        label = i.split("/")
        for lab in label: 
            labels_occurence.append(lab.strip())

In [104]:
count_labels = list()
labels = set(labels_occurence)
for i in list(labels):
    count_labels.append([i,labels_occurence.count(i)])
label_df = pd.DataFrame(columns=["label","appearance_count"],data=count_labels)
label_df.sort_values("appearance_count",ascending=False)["appearance_count"][0:25].sum()

3936

In [106]:
def label_mega_category(row):
    if row["appearance_count"] >= 200:
        row["label_category"] = "Mega Label"
    elif row["appearance_count"] >= 100:
        row["label_category"] = "Great Label"
    elif row["appearance_count"] >= 50:
        row["label_category"] = "Big Label"
    elif row["appearance_count"] >= 20:
        row["label_category"] = "Decent Label"
    elif row["appearance_count"] >= 10:
        row["label_category"] = "Small Label"
    else:
        row["label_category"] = "Mini Label"
    return row
        
label_df = label_df.apply(lambda row: label_mega_category(row),axis=1).sort_values("appearance_count",ascending=False)

We notice also that some songs share multiple labels and that some labels have subdivisions so we propose groupings with the following and re-create categories accordingly

In [105]:
label_df = label_df.sort_values("appearance_count",ascending=False)
labels = label_df["label"]
for label in label_df.index:
    label_stripped = label_df.loc[label,"label"]
    for i in labels:
        stopwords = ["Entertainment","Records","Recordings","LLC","Ltd."]
        i_stripped = i
        for word in stopwords:
            i_stripped = i_stripped.replace(word,"")
            label_stripped = label_stripped.replace(word,"")
        if  fuzz.partial_ratio(label_stripped,i_stripped) >= 90 and fuzz.ratio(label_stripped,i_stripped) >=65:
            if fuzz.ratio(label_df.loc[label,"label"],i) != 100:
                print(label_stripped,i_stripped,fuzz.ratio(label_df.loc[label,"label"],i))
            label_df.loc[label,"label_idx"] = i
            break

Atlantic  Atlantic 67
RCA  Label Nashville RCA  Label 77
Cash Money Cash Money  71
Universal Rec. Universal  84
Interscope  Interscope 71
Def Jam Def Jam  56
Universal Music Universal  69
Universal-Island   Universal  74
Republic Republic  67
Capitol  Nashville Capitol Nashville 81
Motown  Motown 60
Atlantic  UK Atlantic 59
Universal Republic  Universal  79
Sony Music  Sony Music Latin 70
Interscope * Interscope 69
Quality Control Music,  Quality Control Music 89
Nashville Arista Nashville 72
Sony Music UK Sony Music  59
Capitol Capitol  64
Grand Hustle,  Grand Hustle 83
Getting Out Our Dreams, Inc. Getting Out Our Dreams Inc. (G.O.O.D.) Music 75
Virgin EMI Virgin  58
Warner Bros. Warner  69
Geffen* Geffen 92
Rhino Atlantic Atlantic 73
EMI  Nashville Nashville 60
Young Money Young Money Ent. 81
Elektra Elektra  64
Syco Music UK Syco Music 87
Epic  Epic 50
2012 Ameritz Music  Ameritz Music Ltd 85
Cash Money  Inc. Cash Money  88
Capitol  (CAP) Capitol  83
Getting Out Our Dreams II Gettin

In [107]:
label_df2 = label_df.groupby("label_idx").sum().sort_values("appearance_count",ascending=False)

def label_mega_category_groups(row):
    if row["appearance_count"] >= 200:
        row["label_category"] = "Mega Label"
    elif row["appearance_count"] >= 100:
        row["label_category"] = "Great Label"
    elif row["appearance_count"] >= 50:
        row["label_category"] = "Big Label"
    elif row["appearance_count"] >= 20:
        row["label_category"] = "Decent Label"
    elif row["appearance_count"] >= 10:
        row["label_category"] = "Small Label"
    else:
        row["label_category"] = "Mini Label"
    return row
label_df2 = label_df2.apply(lambda row: label_mega_category(row),axis=1).sort_values("appearance_count",ascending=False)

In [118]:
label_df = label_df.join(label_df2,on="label_idx",rsuffix="_idx")

We join the dataframes accordingly 

In [131]:
for song in song_df.index:
    appearance_count = list()
    appearance_count_idx = list()
    label_category = list()
    label_category_idx = list()
    label = song_df.loc[song,"album_label"]
    if type(label) == str:
        label = label.split("/")
        for i in label:
            new_df = label_df[label_df["label"] == i.strip()]
            appearance_count_idx.append(new_df.iloc[0]["appearance_count_idx"])
            label_category_idx.append(new_df.iloc[0]["label_category_idx"])
            label_category.append(new_df.iloc[0]["label_category"])
            appearance_count.append(new_df.iloc[0]["appearance_count"])
        song_df.loc[song,"label_appearance_count"] = max(appearance_count)
        song_df.loc[song,"label_appearance_count_group"] = max(appearance_count_idx)
        song_df.loc[song,"label_category"] = label_category[appearance_count.index(max(appearance_count))]
        song_df.loc[song,"label_category_group"] = label_category_idx[appearance_count_idx.index(max(appearance_count_idx))]

We add a few additional features to our aggregate dataframe related to the number of weeks a song was in the billboard, the first date it appeared on the billboard or if it reached the top 10

In [ ]:
rec_df["join"] = rec_df["artist"] + rec_df["title"]
song_df["join"] = song_df["artist"] + song_df["title"]
rec_df = rec_df.sort_values("date",ascending=False).drop_duplicates(["artist","title"])
rec_df.pop("artist")
rec_df.pop("isnew")
rec_df.pop("rank")
rec_df.pop("date")
rec_df.pop("title")
song_df = song_df.merge(rec_df,on="join")

In [ ]:
song_df["top10"] = song_df["bestrank"] < 10
song_df["top50"] = song_df["bestrank"] < 50
song_df["top25"] = song_df["bestrank"] < 25
song_df["top75"] = song_df["bestrank"] < 75
song_df["top5"] = song_df["bestrank"] < 5

In [162]:
rec_df2 = rec_df.sort_values("date").drop_duplicates(["artist","title"])
rec_df2["join"] = rec_df2["artist"] + rec_df2["title"]
#song_df["join"] = song_df["artist"] + song_df["title"]
rec_df2.pop("artist")
#rec_df2.pop("isnew")
rec_df2.pop("rank")
#rec_df2.pop("date")
rec_df2.pop("title")
rec_df2.pop("bestrank")
rec_df2.pop("firstrank")
#song_df = song_df.merge(rec_df2,on="join")
rec_df2["first_date"] = rec_df2["date"]
rec_df2.pop("date")
song_df = song_df.merge(rec_df2,on="join")

113199    100
113126     27
113127     28
113128     29
113129     30
         ... 
90         91
99        100
78         79
54         55
69         70
Name: firstrank, Length: 8519, dtype: int64

We transform the dates into datetime object and save the year and the month as new features so as to account for the seasonality/trends 

In [178]:
song_df["datetime"] = song_df["first_date"].apply(lambda x: datetime.strptime(x.strip(),"%Y-%m-%d") if len(str(x))>8 else x)
song_df["datetime_year"] = song_df["datetime"].apply(lambda x: x.year)
song_df["datetime_month"] = song_df["datetime"].apply(lambda x: x.month)

In [179]:
song_df["release_month"] = song_df["spotify_album_release_date_datetime"].apply(lambda x: x.month)
song_df["release_year"] = song_df["spotify_album_release_date_datetime"].apply(lambda x: x.year)

In [181]:
with open('song_df_aggregate.pkl', 'wb') as f:
    pickle.dump(song_df, f)